# Data Cleaning Notebook

This notebook contains code to clean and consolidate the extremely messy pricing files of US hospitals, which were extracted from hospital websites in a very large scraping exercise. There are many steps required to clean the files, which are each contained in different sections below. The files contained in `checked_files`, which can be dowloaded from [this dropbox link](https://www.dropbox.com/sh/afpoveqn7elmzkn/AACTMyNvGZ3ZtCe08ualKrRba?dl=0), have already been checked to be "openable" and to likely be pricing files. There are roughly x separate discrete steps - - each of which are described below.

## 1. Find Unique Files

The scraping exercise would often download the same file multiple times because 1) files are often available at multiple addresses and 2) our crawling procedure found the same price file/url for multiple hospitals. The code below attempts to identify the unique files within each hospital folder and returns their paths. Note that, because of 2) above, we'll eventually need to check across hospitals to ensure that the same file isn't being used for multiple hospitals. For memory reasons, this will likely require a comparison based on file object rather than opened dataframe contents. See [this Stack Exchange post](https://stackoverflow.com/questions/748675/finding-duplicate-files-and-removing-them) for ideas.

In [1]:
def unique_filecheck(df_list, path_list): 
    """
    Returns the unique dataframes and their paths
    from an initial list of dataframes and paths
    """
  
    unique_df_list, unique_path_list = [], []
      
    # traverse for all elements 
    for dataframe, path in zip(df_list, path_list): 
        
        check = [dataframe.equals(x) for x in unique_df_list]
        if np.sum(check) == 0:
            unique_df_list.append(dataframe)
            unique_path_list.append(path)
    
    return(unique_path_list)

def get_unique_paths(home_folder):
    """
    Returns list of paths to unique files within
    each hospital sub-folder of the home_folder
    """
    
    unique_dfs, unique_paths = [], []
    
    # Get list of hospitals in folder
    folders = os.listdir(home_folder)

    # Loop over each hospital checking for its unique files
    for folder in folders:
    
        # First compile all dataframes and their paths into lists
        df_list, path_list = [], []
        files = os.listdir(home_folder + folder)
        for file in files:

            fname = home_folder + folder + "/" + file

            if ".csv" in file:

                # Read file
                df = pd.read_csv(fname,index_col=False)

                # Add to list
                df_list.append(df)
                path_list.append(fname)

            if ".xls" in file:

                # Read file
                df = pd.read_excel(fname,index_col=False)

                # Add to list
                df_list.append(df)
                path_list.append(fname)
        
        # Second, find the unique dataframes and their lists for that hospital
        unique_paths += unique_filecheck(df_list,path_list)
        
    return(unique_paths)

## 2. Open Clean DataFrames

The pricing files come in an enormous variety of formats. For example, not all header rows are in the first row of the file, there are often notes or introductory text in the files, there are blank columns, variables names are inconsistent etc. The code below attempts to deal with these various issues. The first function searches for the location of the headers within a file. The second function uses this information to open the file into a clean dataframe format.

In [2]:
def find_header_row(df):
    """
    Returns the row within a file that is likely the header.
    """
    
    # Create FALSE flag when cell has NAN entry
    df_flag = ~df.isnull()
    
    # Loop over rows to find last row of all NANs. The row below that will be the header row.
    # We also make sure that there were just a few NA's above it. This takes care of the files
    # with notes at the bottom.
    headrow, consecutive_nonNA_rows = 0, 0
    for index, row in df_flag.iterrows():
        consecutive_nonNA_rows += 1
        if (sum(row) == 0) & (consecutive_nonNA_rows < 40):
            headrow, consecutive_nonNA_rows = index + 2, 0

    return(headrow)

def open_file(fname):
    """
    Returns a pandas dataframe from a file name.
    Takes care of various formatting issues.
    """
    
    if ".csv" in fname:
            
        # Read file
        df = pd.read_csv(fname,index_col=False)
        d = pd.read_csv(fname,index_col=False,skiprows=find_header_row(df))
            
    if ".xls" in fname:
   
        # Read file
        df = pd.read_excel(fname,index_col=False)
        d = pd.read_excel(fname,index_col=False,skiprows=find_header_row(df))
    
    # Remove Unnamed columns
    droplist = [i for i in d.columns if i.startswith('Unnamed')]
    d.drop(droplist,axis=1,inplace=True)
    
    return(d)

## 3. Explore Variable/Header Names

Pricing files generally assign each price either to a text description of a service/product, or to a code representing that service/product. When merging prices across hospitals, we want be able to distinguish price, description and code columns, so that we can make sure that we are merging dataframes on the right variables. The first function below extracts all unique header labels from all the files specified within a list. The idea is that we can manually inspect this information to come up with good keywords to identify the appropriate columns. The next function then uses these keywords to create two lists - one with all the unique text descriptive labels from the datasets and the other with all the unique codes, which we can use to identify the columns in the section below.

In [3]:
def get_header_labels(unique_paths):
    """
    Returns all unique headersfrom a set of
    files defined in `unique_paths`.
    """
    headers = []
    for file in unique_paths:
        df = open_file(file)
        headers += list(df.columns.values)

    # Print to manually inspect header values in order to group the below
    return(Counter(headers))

def get_descriptive_labels(unique_paths,description_kws,code_kws):
    """
    Returns lists of unique code and description labels defined by
    `description_kws` and `code_kws` from a set of files defined
    in `unique_paths`.
    """
    
    descriptions, codes = [], []
    for file in unique_paths:

        df = open_file(file)

        # Get descriptive columns
        description_cols = [i for i in df.columns if any(x in i.lower() for x in description_kws)]
        if  description_cols:
            d = df[description_cols]
            descriptions += d.iloc[:,0].tolist()

        # Get code columns
        code_cols = [i for i in df.columns if any(x in i.lower() for x in code_kws)]
        if code_cols:
            d = df[code_cols]
            codes += d.iloc[:,0].tolist()
    
    descriptions, codes = list(set(descriptions)), list(set(codes))
    
    return([x for x in descriptions if str(x) != 'nan'],[x for x in codes if str(x) != 'nan'])

## 4. Merge Files Based on Exact Description and/or Code Match

As discussed, we need to merge the pricing files based either on their product/service text descriptions and/or their product/service codes. The first function below creates a merged dataframe using all the files in `unique_paths` that have columns identified in the `index_kws` list. The merging is based on exact matches of the contents of the `index_kws` column. Note that we should improve this by using fuzzy matching at some point. The next function attempts to map text descriptions to codes based on files that happen to contain both. 

In [4]:
def merge_data(unique_paths,price_kws,index_col,index_kws):
    """
    Returns a merged pricing file of all files in `unique_paths`
    that have columns with keywords in `index_kws` and `price_kws`
    and merges based on matches in the columns identified using
    `index_kws`.
    """
    
    # Initiate merged dataframe
    d_merged = pd.DataFrame({'desc':index_col})
    d_merged.set_index('desc',inplace=True)

    # Loop over files, extract relevant columns and merge
    for file in unique_paths:

        # Open file
        print(file)
        df = open_file(file)

        # Get price columns (choose the last column if multiple)
        price_cols = [i for i in df.columns if any(x in i.lower() for x in price_kws)]
        price_cols = price_cols[-1:]   

        # Get descriptive/code columns
        desc_cols = [i for i in df.columns if any(x in i.lower() for x in index_kws)]

        if (len(price_cols) > 0) & (len(desc_cols) > 0):

            # Extract relevant columns from dataframe
            d = df.loc[:,[desc_cols[0],price_cols[0]]]

            # Remove '$' sign from price column
            d.loc[:,price_cols[0]] = d.loc[:,price_cols[0]].replace('[\$,]', '', regex=True)
            d.loc[:,price_cols[0]] = pd.to_numeric(d.loc[:,price_cols[0]],errors='coerce')

            # Take average when there are multiple rows with same label
            d = d.groupby(desc_cols[0]).mean()

            # Merge to dataframe
            d_merged = d_merged.merge(d,left_index=True, right_index=True,how='left')

            # Rename Price Column to Hospital Name
            hospital_name = file.split("/")[2]
            d_merged.rename(columns={price_cols[0]:hospital_name},inplace=True)
    
    return(d_merged)

def match_desc_code(unique_paths,desc_col,description_kws,code_kws):
    """
    Returns a dataframe mapping text descriptions to codes.
    """
    # Initiate merged dataframe
    d_desc_code = pd.DataFrame({'desc':desc_col})
    d_desc_code.set_index('desc',inplace=True)

    # Loop over files, extract relevant columns and merge
    for file in unique_paths:

        # Open file
        df = open_file(file)
        print(file)

        # Get description columns
        desc_cols = [i for i in df.columns if any(x in i.lower() for x in description_kws)]

        # Get code columns
        code_cols = [i for i in df.columns if any(x in i.lower() for x in code_kws)]

        if (len(code_cols) > 0) & (len(desc_cols) > 0):

            # Extract relevant columns from dataframe
            d = df.loc[:,[desc_cols[0],code_cols[0]]]
            d.set_index(desc_cols[0],inplace=True)

            # Merge to dataframe
            d_desc_code = d_desc_code.merge(d,left_index=True, right_index=True,how='left')

    # Drop duplicates
    d_desc_code.drop_duplicates(keep = 'first', inplace = True) 

    # Stack the columns and the index
    df_idx = pd.DataFrame({'desc':d_desc_code.index})
    df_idx = pd.concat([df_idx]*len(d_desc_code.columns),ignore_index=True)
    df_idx['code'] = sorted(it.chain(*d_desc_code.values))

    # Clean up df
    df_idx.dropna(inplace = True)
    df_idx = df_idx.drop_duplicates(keep = 'first').reset_index(drop=True)

    return(df_idx)

## 5. Run Analyses

The code below runs the functions defined above to compile a merged dataframe of hospital prices. We start with just Washington DC hospitals, which are [here](https://www.dropbox.com/sh/32pjpd7ix00wg8m/AABEWzJirpWalstZGHyBaiKHa?dl=0), to make sure it runs properly.

In [5]:
# -------------------
# Housekeeping
# -------------------

import pandas as pd
import numpy as np
from collections import Counter
import os
import itertools as it

# Set location of pricing files
home_folder = "../DC_hospitals/"
save_fname = "DC_merge.csv"

# -------------------
# Summarize
# -------------------

# Number of hospitals
folders = os.listdir(home_folder)
print("Number of Hospitals: " + str(len(folders)))

# Number of files
FILES = []
for folder in folders:
    FILES += os.listdir(home_folder + folder)
files = [x for x in FILES if x != ".DS_Store"]
print("Total Number of Files: " + str(len(files)))

# -------------------
# Find unique files
# -------------------

unique_paths = get_unique_paths(home_folder)

# -------------------
# Inspect header labels & extract unique values
# -------------------

# Compile list of all the header labels
labels = get_header_labels(unique_paths)
print(labels)

# Manually define keywords based on above results
description_kws, code_kws, price_kws = ['description','name'],['code',"drg"],["chg","price","charge"]

# Get unique values for description/code columns
desc_col, code_col = get_descriptive_labels(unique_paths,description_kws,code_kws)

# -------------------
# Merge Data
# -------------------

# Merge data based on description match
d_desc = merge_data(unique_paths,price_kws,desc_col,description_kws)

# Merge data based on code match
d_code = merge_data(unique_paths,price_kws,code_col,code_kws)

# Get mapping between codes and descriptions
d_match = match_desc_code(unique_paths,desc_col,description_kws,code_kws)

# -------------------
# Merge Descriptions onto Code Data
# -------------------

# Get matching dataframe and remove duplicates from code column
d1 = d_match.copy().drop_duplicates(subset ="code",keep="first")
d1.set_index('code',inplace=True)

# Merge to dataframe
d_codebased = d_code.copy().merge(d1,left_index=True, right_index=True,how='left')

# -------------------
# Merge Codes onto Descriptions Data
# -------------------

# Get matching dataframe and remove duplicates from description column
d1 = d_match.copy().drop_duplicates(subset ="desc",keep="first")
d1.set_index('desc',inplace=True)

# Merge to dataframe
d_descbased = d_desc.copy().merge(d1,left_index=True, right_index=True,how='left')

# -------------------
# Merge into final dataset
# -------------------

d_final = d_descbased.groupby(by=d_descbased.columns, axis=1).mean().round(decimals=2)

# Rearrange Columns
d_final=d_final.reset_index()
cols = d_final.columns.tolist()
cols = cols[-1:] + cols[:-1]
d_final = d_final[cols]

# Rename Columns
d_final.rename(columns={'code':'Code','desc':'Description'},inplace=True)

# Save
d_final.to_csv(save_fname)

Number of Hospitals: 7
Total Number of Files: 21
Counter({'PROCEDURE NAME': 5, 'UNIT CHG AMT.': 5, 'SVC CODE': 4, 'SERVICES DESCRIPTION': 4, 'CHGAMT': 4, 'Price': 4, 'Procedure Description': 2, 'Description': 2, 'FEE SCHED ID': 1, 'FEE SCHEDULE NAME': 1, 'PROC CODE': 1, 'CPT CODE': 1, 'CPT EFF. DATE': 1, 'UB CODE': 1, 'MSDRG': 1, 'DRG Name': 1, 'Avg Charge': 1, 'Charge Description': 1, 'Standard Charge': 1, 'DRG': 1, 'MS-DRG Description': 1, 'Average of Chgs': 1})
../DC_hospitals/SIBLEY MEMORIAL HOSPITAL/4.xlsx
../DC_hospitals/SIBLEY MEMORIAL HOSPITAL/8.xlsx
../DC_hospitals/SIBLEY MEMORIAL HOSPITAL/7.xls
../DC_hospitals/SIBLEY MEMORIAL HOSPITAL/6.xls
../DC_hospitals/SIBLEY MEMORIAL HOSPITAL/5.xls
../DC_hospitals/MEDSTAR GEORGETOWN UNIVERSITY HOSPITAL/1.csv
../DC_hospitals/MEDSTAR GEORGETOWN UNIVERSITY HOSPITAL/0.xlsx
../DC_hospitals/MEDSTAR WASHINGTON HOSPITAL CENTER/0.xls
../DC_hospitals/CHILDREN'S HOSPITAL NMC/0.csv
../DC_hospitals/PROVIDENCE HOSPITAL WASHINGTON DC/0.csv
../DC_hospit

## 6. Display Merged DataSet

The first few rows of the resulting dataset are displayed below.

In [6]:
d_final.head(n=40)

,Code,Description,CHILDREN'S HOSPITAL NMC,HOWARD UNIVERSITY HOSPITAL,MEDSTAR GEORGETOWN UNIVERSITY HOSPITAL,MEDSTAR WASHINGTON HOSPITAL CENTER,PROVIDENCE HOSPITAL WASHINGTON DC,SIBLEY MEMORIAL HOSPITAL,UNITED MEDICAL CENTER
0,NaN,GFT GELWEAVE THOR ARCH 12X10X8,NaN,NaN,NaN,6926.49,NaN,NaN,NaN
1,NaN,CATH STEER 6F LG 8 ELECTRD LF,NaN,NaN,NaN,NaN,NaN,984.75,NaN
2,NaN,SOLN DIANEAL 2500L-CAL,NaN,NaN,NaN,47.75,NaN,NaN,NaN
3,NaN,CABLE HOMOG 1.6MM VIT 29.5,NaN,NaN,NaN,NaN,NaN,1223.73,NaN
4,NaN,BOLT LOCK 3.9MMX78MM BLUE,NaN,NaN,NaN,NaN,NaN,920.90,NaN
5,NaN,*TROPICAMIDE OPTH 1% SOL 2ML,NaN,NaN,NaN,6.07,NaN,NaN,NaN
6,NaN,SCR 5.5X45MM,NaN,NaN,NaN,3418.37,NaN,NaN,NaN
7,NaN,PLATE DISTAL FIBULA 2.7MM 7H L,NaN,NaN,NaN,NaN,NaN,4508.32,NaN
8,NaN,SCREW LOCKING T10 2.7*12MM,NaN,NaN,NaN,NaN,NaN,2366.89,NaN
9,NaN,SET OATS RETROGRADE STER 8MM,NaN,NaN,NaN,2801.41,NaN,NaN,NaN


## 7. To Do List

1. **Fuzzy matching**. We have so far matched datasets based on exact matches of text descriptions or codes. But, as displayed above, this results in many blank cells. This is likely because hospitals define their services slightly differently. A fuzzy matching procedure would help to over come this. Take a look at [this blog post](https://medium.com/@categitau/fuzzy-string-matching-in-python-68f240d910fe) for ideas.
2. **Use official codebooks for matching**. The mapping between text descriptions and codes has so far been done by exploiting pricing files that have both. We could instead use [official codebooks](https://www.findacode.com/drg/drg-diagnosis-related-group-codes.html) to perform this matching.
3. **Identify unique files across all hospitals**. We have so far identified duplicates within a hospital's files by opening them and comparing the resulting dataframes. We need to do this across all hospitals, which will probably require a direct file comparison using techniques in [this post](https://stackoverflow.com/questions/748675/finding-duplicate-files-and-removing-them).
4. **Ensure files are properly assigned**. For those that are duplicates, and for those hospitals with multiple price files, we need to determine whether they are correctly assigned to that hospital. For example, take a look at the [Sibley Hospital folder](https://www.dropbox.com/sh/e6au1phdxb6pdjn/AACw38q-M47wssX20I7y5NDra?dl=0), which contains multiple price files relating to other hospitals. A keyword search of hospital names would suffice in this case to distinguish the files.